import modules & 

In [1]:
import pandas as pd
from datetime import date, timedelta
import datetime
import holidays
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: c:\apps\Project1\mid_code


load data

In [2]:
jpy = pd.read_csv('../dataset/JPY_KRW.csv')[['날짜','종가','시가']].rename(
    columns={'날짜':'date',
             '종가':'close',
             '시가':'open'})

In [3]:
# 날짜순으로 오름차순 정렬
jpy.sort_values(by='date',inplace=True)

In [4]:
print(jpy.info(), jpy.head(), jpy.tail())

<class 'pandas.core.frame.DataFrame'>
Index: 66 entries, 65 to 0
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    66 non-null     object 
 1   close   66 non-null     float64
 2   open    66 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.1+ KB
None             date   close    open
65  2023- 07- 28  9.0157  9.2085
64  2023- 07- 31  8.9794  9.0303
63  2023- 08- 01  8.9925  8.9833
62  2023- 08- 02  9.0625  8.9957
61  2023- 08- 03  9.1114  9.0666            date   close    open
4  2023- 10- 23  8.9636  9.0147
3  2023- 10- 24  8.9640  8.9736
2  2023- 10- 25  9.0007  8.9674
1  2023- 10- 26  8.9934  9.0070
0  2023- 10- 27  9.0101  8.9875


In [5]:
# 한국의 공휴일 가져오기
kr_holidays = holidays.SouthKorea(years=2023)

# 기간 설정 (예: 2023년 1월 1일부터 12월 31일까지)
start_date = date(2023, 7, 28)
end_date = date(2023, 10, 20)

# 한국의 공휴일을 제외한 날짜 리스트 생성
date_list = []
current_date = start_date
while current_date <= end_date:
    if current_date in kr_holidays:
        date_list.append(current_date)
    current_date += timedelta(days=1)
date_list

[datetime.date(2023, 8, 15),
 datetime.date(2023, 9, 28),
 datetime.date(2023, 9, 29),
 datetime.date(2023, 9, 30),
 datetime.date(2023, 10, 3),
 datetime.date(2023, 10, 9)]

In [6]:
jpy['date'] = pd.to_datetime(jpy['date'])
jpy = jpy.loc[~jpy['date'].isin(date_list)]
jpy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 65 to 0
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    61 non-null     datetime64[ns]
 1   close   61 non-null     float64       
 2   open    61 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.9 KB


In [7]:
jpy.head(22)

,date,close,open
65,2023-07-28,9.0157,9.2085
64,2023-07-31,8.9794,9.0303
63,2023-08-01,8.9925,8.9833
62,2023-08-02,9.0625,8.9957
61,2023-08-03,9.1114,9.0666
60,2023-08-04,9.1979,9.1195
59,2023-08-07,9.1721,9.2001
58,2023-08-08,9.1815,9.1793
57,2023-08-09,9.1357,9.1855
56,2023-08-10,9.0971,9.1393


#### 환율 변동값 전처리
* 전일 변동율 = (전날종가-전날시가)/전날시가 yesterday_change <br>
* 야간 변동율 = (당일시가-전날종가)/전날종가 night_change <br>
* 데이터 기간 : 2023/07/31 ~ 2023/10/20

In [8]:
# 파생변수 생성 (전일 변동율 = yesterday_change)
jpy['jpy_yesterday_change'] = round((jpy['close']-jpy['open'])/jpy['open']*100, 2)
jpy['jpy_yesterday_change'] = jpy['jpy_yesterday_change'].shift(1)

# 파생변수 생성(야간 변동율 = night_change)
jpy['close_shift'] = jpy['close'].shift(1)
jpy['jpy_night_change'] = round((jpy['open']-jpy['close_shift'])/jpy['close_shift']*100,2)

jpy.drop('close_shift',axis=1,inplace=True)

In [9]:
jpy.head(22)

,date,close,open,jpy_yesterday_change,jpy_night_change
65,2023-07-28,9.0157,9.2085,NaN,NaN
64,2023-07-31,8.9794,9.0303,-2.09,0.16
63,2023-08-01,8.9925,8.9833,-0.56,0.04
62,2023-08-02,9.0625,8.9957,0.10,0.04
61,2023-08-03,9.1114,9.0666,0.74,0.05
60,2023-08-04,9.1979,9.1195,0.49,0.09
59,2023-08-07,9.1721,9.2001,0.86,0.02
58,2023-08-08,9.1815,9.1793,-0.30,0.08
57,2023-08-09,9.1357,9.1855,0.02,0.04
56,2023-08-10,9.0971,9.1393,-0.54,0.04


In [10]:
print(jpy.info(), jpy.head(), jpy.tail())

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 65 to 0
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  61 non-null     datetime64[ns]
 1   close                 61 non-null     float64       
 2   open                  61 non-null     float64       
 3   jpy_yesterday_change  60 non-null     float64       
 4   jpy_night_change      60 non-null     float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 2.9 KB
None          date   close    open  jpy_yesterday_change  jpy_night_change
65 2023-07-28  9.0157  9.2085                   NaN               NaN
64 2023-07-31  8.9794  9.0303                 -2.09              0.16
63 2023-08-01  8.9925  8.9833                 -0.56              0.04
62 2023-08-02  9.0625  8.9957                  0.10              0.04
61 2023-08-03  9.1114  9.0666                  0.74              0.05         date 

In [11]:
# 데이터 기간 : 2023/07/31 ~ 2023/10/20
jpy = jpy.loc[jpy['date']<='2023-10-20',].loc[jpy['date']>='2023-07-31',]
jpy = jpy[['date','jpy_yesterday_change','jpy_night_change']]
jpy.reset_index(drop=True, inplace=True)

In [12]:
jpy.head(22)

,date,jpy_yesterday_change,jpy_night_change
0,2023-07-31,-2.09,0.16
1,2023-08-01,-0.56,0.04
2,2023-08-02,0.10,0.04
3,2023-08-03,0.74,0.05
4,2023-08-04,0.49,0.09
5,2023-08-07,0.86,0.02
6,2023-08-08,-0.30,0.08
7,2023-08-09,0.02,0.04
8,2023-08-10,-0.54,0.04
9,2023-08-11,-0.46,0.05


In [13]:
print(jpy.info(), jpy.head(), jpy.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  55 non-null     datetime64[ns]
 1   jpy_yesterday_change  55 non-null     float64       
 2   jpy_night_change      55 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.4 KB
None         date  jpy_yesterday_change  jpy_night_change
0 2023-07-31                 -2.09              0.16
1 2023-08-01                 -0.56              0.04
2 2023-08-02                  0.10              0.04
3 2023-08-03                  0.74              0.05
4 2023-08-04                  0.49              0.09          date  jpy_yesterday_change  jpy_night_change
50 2023-10-16                  0.37              0.12
51 2023-10-17                 -0.41              0.04
52 2023-10-18                  0.13              0.05
53 2023-10

csv 파일로 새로 저장

In [14]:
jpy.to_csv("../dataset/clean_feature/jpy_krw_final.csv")